In [2]:
import random
import csv
import numpy as np
import pandas as pd
import Loader
import tensorflow as tf
import Transformer
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import MeanSquaredError
#use for dark mode
plt.rcParams["font.family"] = "serif"
plt.style.use('dark_background')

In C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Tim\anaconda3\envs\tensorflow\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Tim\anaconda3\envs\tenso

In [6]:
def runBasicSVM(Xtrain, Ytrain, valData):
    maxC=10**-.01
    clf=svm.SVC(
        C=maxC,                          # The regularization parameter
        kernel='rbf',                   # The kernel type used 
        degree=4,                       # Degree of polynomial function 
        gamma='scale',                  # The kernel coefficient
        coef0=0.0,                      # If kernel = 'poly'/'sigmoid'
        shrinking=True,                 # To use shrinking heuristic
        probability=False,              # Enable probability estimates
        tol=0.001,                      # Stopping crierion
        cache_size=200,                 # Size of kernel cache
        class_weight=None,              # The weight of each class
        verbose=False,                  # Enable verbose output
        max_iter=- 1,                   # Hard limit on iterations
        decision_function_shape='ovr',  # One-vs-rest or one-vs-one
        break_ties=False,               # How to handle breaking ties
        random_state=None               # Random state of the model
    )
    #Xtrain,_,valData = Transformer.normalizeData(Xtrain, compoundDataTest, compoundDataValidate, newMean=0, newStd=1)
    clf.fit(Xtrain, Ytrain)
    predictions = clf.predict(Xtrain)
    val_pred = clf.predict(valData)
    return [predictions, val_pred]

def get_data():
    compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
    compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
    compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

    labelsMean, trainMean = Transformer.useAverageFD(labelsTrain, compoundDataTrain)
    _, testMean = Transformer.useAverageFD(labelsTest, compoundDataTest)
    _, valMean = Transformer.useAverageFD(labelsValidate, compoundDataValidate)

    labelsMax, trainMax = Transformer.useMaxFD(labelsTrain, compoundDataTrain)
    _, testMax = Transformer.useMaxFD(labelsTest, compoundDataTest)
    _, valMax = Transformer.useMaxFD(labelsValidate, compoundDataValidate)

    #after transformations are done assign data
    dataLabels = labelsMax
    trainData = trainMax
    testData = testMax
    valData = valMax

    trainData, testData, valData = Transformer.normalizeData(trainData, testData, valData, newMean=0, newStd=1)
    
    classTrain = Transformer.toBinaryClassification(activitiesTrain)
    classVal = Transformer.toBinaryClassification(activitiesValidate)
    classTest = Transformer.toBinaryClassification(activitiesTest)
    
    return trainData, testData, valData, classTrain, classTest, classVal, dataLabels


def parameter_rank(top_x, epochs, avg_std_runs):
    
    trainData, testData, valData, classTrain, classTest, classVal, dataLabels = get_data()
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(trainData,classTrain,
                                       random_state=104, 
                                       test_size=0.20, 
                                       shuffle=True)
    col_idx = np.arange(0,210)
    probs = []
    for i in range(210):
        probs.append(float(50))
    probs = np.array(probs)
    eval_set = []

    for x in range(epochs):
        
        train_data_df = pd.DataFrame(X_train, columns = dataLabels)
        train_temp_df = train_data_df.copy()
        val_data_df = pd.DataFrame(X_test, columns = dataLabels)
        val_temp_df = val_data_df.copy()

        values = probs
        
        arr1 = values / values.min()
        arr1 = arr1 / arr1.sum()
        #print(arr1, arr1.sum())
        #display(values, sum(values))
        test_cols = np.random.choice(col_idx, top_x, replace=False, p = arr1)
        # using set() to perform task
        test_cols_set = set(test_cols)
        col_idx_set = set(col_idx)
        zero_cols = list(col_idx_set - test_cols_set)
        
        train_temp_df.iloc[:, zero_cols] *= 0
        val_temp_df.iloc[:, zero_cols] *= 0
        
        
        Xtrain = train_temp_df.values
        Xval = val_temp_df.values
        accuracy = runBasicSVM(Xtrain,y_train, Xval)
        curr_val_acc = accuracy_score(y_test, accuracy[1])
        
        if(x < avg_std_runs):
            eval_set.append(curr_val_acc)
        else:
            
            if curr_val_acc > np.average(eval_set) + np.std(eval_set):
                np.multiply.at(probs,test_cols,1.1)
            elif curr_val_acc < np.average(eval_set) - np.std(eval_set):
                np.multiply.at(probs,test_cols,.9)
                prev_val_acc = accuracy[1]
            else:
                prev_val_acc = accuracy[1]
                
            eval_set.pop(0)
            eval_set.append(curr_val_acc)
        
        if((((x+1)/epochs)*100)%10 == 0):
            print(((x+1)/epochs)*100, "% complete")
            #print(probs)

    top_idx = np.flip(probs.argsort()[-210:])
    #display(final_probs,final_probs.argsort()[-5:])
    res_probs_list = [probs[i] for i in top_idx]
    res_feats_list = [dataLabels[i] for i in top_idx]
    result = [res_feats_list, res_probs_list]
    #display(results, results.shape)
    rslt_df = pd.DataFrame(data = np.transpose(result), columns=['Feature','Probability Weight'])
    #sorted_rslt_df = rslt_df.sort_values(by='Probability Weight', ascending=False)
    
    train_data_df = pd.DataFrame(trainData, columns = dataLabels)
    train_temp_df = train_data_df.copy()
    val_data_df = pd.DataFrame(valData, columns = dataLabels)
    val_temp_df = val_data_df.copy()
    
    top_50_idx = probs.argsort()[-50:]
    #test_cols = np.random.choice(col_idx, top_50_idx, replace=False, p = arr1)
    # using set() to perform task
    test_cols_set = set(top_50_idx)
    col_idx_set = set(col_idx)
    zero_cols = list(col_idx_set - test_cols_set)
    
    train_temp_df.iloc[:, zero_cols] *= 0
    val_temp_df.iloc[:, zero_cols] *= 0


    Xtrain = train_temp_df.values
    Xval = val_temp_df.values
    accuracy = runBasicSVM(Xtrain,classTrain, Xval)
    in_samp_acc = accuracy_score(classTrain, accuracy[0])
    out_samp_acc = accuracy_score(classVal, accuracy[1])
    
    final_acc = [in_samp_acc, out_samp_acc]
    
    

    return rslt_df, probs, dataLabels, final_acc
#for x in rslt
results_df, final_probs, features, final_acc = parameter_rank(50,300,20)
display(results_df, final_acc)
        

10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete
60.0 % complete
70.0 % complete
80.0 % complete
90.0 % complete
100.0 % complete


,Feature,Probability Weight
0,peoe_vsa12,419.3315715011778
1,fr_aniline,324.69374507667493
2,slogp_vsa12,295.17613188788624
3,peoe_vsa9,260.3719141769857
4,fr_nh1,246.4113297336056
...,...,...
205,fr_term_acetylene,23.438939584500012
206,bcut2d_mrlow,23.204550188655
207,smr_vsa8,22.51535184350178
208,fr_arn,20.884095169789507


[0.9503128450496872, 0.9205882352941176]

In [9]:
# with pd.option_context('display.max_rows', None,'display.max_columns', None,
#     'display.precision', 3,):
#     display(results)
print("In Sample ACC: ", final_acc[0])
print("Out of Sample ACC: ", final_acc[1])
display("Top 50 Features",results_df.head(50), "Bottom 50 Features", results_df.tail(50))

In Sample ACC:  0.9503128450496872
Out of Sample ACC:  0.9205882352941176


'Top 50 Features'

,Feature,Probability Weight
0,peoe_vsa12,419.3315715011778
1,fr_aniline,324.69374507667493
2,slogp_vsa12,295.17613188788624
3,peoe_vsa9,260.3719141769857
4,fr_nh1,246.4113297336056
5,fpdensitymorgan1,209.87935373793712
6,balabanj,209.87935373793707
7,estate_vsa9,205.70275459855202
8,smr_vsa4,185.13247913869688
9,smr_vsa9,164.953038912579


'Bottom 50 Features'

,Feature,Probability Weight
160,chi0n,40.697193375396246
161,fr_guanido,40.69719337539624
162,peoe_vsa8,39.887319227225866
163,peoe_vsa7,39.488446034953604
164,fr_coo,38.90413840500002
165,fr_aldehyde,38.51509702095003
166,fr_priamide,38.51509702095003
167,fr_barbitur,37.74864659023313
168,kappa1,37.748646590233115
169,fr_phenol_noorthohbond,37.37116012433081


# Push the limits

In [38]:
#for x in range(5):
_, _, _, _, _, _, full_features = get_data() 
top_50 = results_df.head(50)
top_50_features = top_50['Feature']
#print(feature, full_features)
full_feature_TF = np.isin(full_features, top_50_features)
#print(np.isin(full_features, top_50_features))
#col_idx = np.where(full_feature_TF == True)[0]
print(col_idx, len(col_idx))
remove = [10, 20, 30, 40, 45]
for x in remove:
    
    temp_col_idx = col_idx.copy()
    end = 50 - x
    idx = np.arange(end, 50)
    temp_col_idx = np.delete(temp_col_idx, idx)
    #print(temp_col_idx, len(temp_col_idx))
    
    

[  0   1   8  11  16  17  22  23  27  36  42  45  50  51  60  65  66  70
  71  74  79  80  82  88  94  96 107 108 112 113 114 116 123 129 130 131
 141 142 144 150 154 157 161 164 170 182 187 193 195 201] 50
[  0   1   8  11  16  17  22  23  27  36  42  45  50  51  60  65  66  70
  71  74  79  80  82  88  94  96 107 108 112 113 114 116 123 129 130 131
 141 142 144 150] 40
[  0   1   8  11  16  17  22  23  27  36  42  45  50  51  60  65  66  70
  71  74  79  80  82  88  94  96 107 108 112 113] 30
[ 0  1  8 11 16 17 22 23 27 36 42 45 50 51 60 65 66 70 71 74] 20
[ 0  1  8 11 16 17 22 23 27 36] 10
[ 0  1  8 11 16] 5


In [ ]:
test_in_acc = []
test_out_acc = []
for x in range(10):
    results_df_100, final_probs_100, features_100, final_acc_100 = parameter_rank(50,100,20)
    test_in_acc.append(final_acc_100[0])
    test_out_acc.append(final_acc_100[1])

display(test_in_acc, np.mean(test_in_acc), test_out_acc, np.mean(test_out_acc))
feat_100 = results_df_100.head(50)
feat_300 = results_df.head(5)


#print(feat_100['Feature'].values == feat_300['Feature'].values)

10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete
60.0 % complete
70.0 % complete
80.0 % complete
90.0 % complete
100.0 % complete
10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete


In [ ]:
display(np.isin(feat_300['Feature'].values, feat_100['Feature'].values))
#display(feat_100, feat_300)

display(final_acc_100)